In [35]:
import numpy as np
import pandas as pd
from pandas import DataFrame, read_csv
from bokeh.plotting import figure
from bokeh.io import output_notebook, show, output_file, curdoc

def clean_data(df):    
    columns_list = list(df.columns)
    df_incoming = df.loc[df['direction']=='Incoming']
    df_outgoing = df.loc[df['direction']=='Outgoing']
    df_outgoing_columns = list(df_outgoing)
    #print (df_outgoing.tail())
    df_outgoing_columns[0],df_outgoing_columns[1] = df_outgoing_columns[1],df_outgoing_columns[0]
    df_temp = df_outgoing.loc[:,df_outgoing_columns]
    df_outgoing = df_temp
    df_outgoing_columns[0],df_outgoing_columns[1] = df_outgoing_columns[1],df_outgoing_columns[0]
    df_outgoing.columns = df_outgoing_columns

    del (df_temp)
    bigdf = df_incoming.append(df_outgoing,ignore_index='True')
    bigdf = bigdf.drop('direction',axis=1)
    del(df_incoming,df_outgoing)
    return bigdf

df = pd.read_csv('csv\calls.csv')
raw_df = df
df = clean_data(df)
#print (df.head())

In [42]:
from pandas.api.types import is_string_dtype
from pandas.api.types import is_numeric_dtype

print (is_numeric_dtype(df['timestamp']))

print (df.head())

False
         user        other  duration                       timestamp
0  7610039694  07434677419       211  Wed Sep 15 19:17:44 +0100 2010
1  7641036117  07371326239        45  Mon Feb 11 07:45:42 +0000 1980
2  7641036117  07681546436         0  Mon Feb 11 08:06:18 +0000 1980
3  7641036117  07588304495       124  Thu Sep 09 19:35:37 +0100 2010
4  7163185791  07230262224        47  Tue Sep 14 15:28:16 +0100 2010
